# Анализ игрового рынка (2000-2013 гг.) для исследования развития индустрии и жанра RPG

- Автор: Майорова Алина Евгеньевна
- Дата: 17.09.2025

### Цели и задачи проекта

Основная цель данного проекта заключается в комплексной предобработке исторических данных о компьютерных играх для последующего анализа развития игровой индустрии в период с 2000 по 2013 год. В рамках поставленной задачи необходимо выполнить полный цикл работы с данными: начиная с первичного знакомства и проверки их корректности, заканчивая формированием целевого среза данных, готового для глубокого анализа.

### Описание данных

В проекте будут использованы данные из файла `/datasets/new_games.csv` с таким описанием:
- Name — название игры.
- Platform — название платформы.
- Year of Release — год выпуска игры.
- Genre — жанр игры.
- NA sales — продажи в Северной Америке (в миллионах проданных копий).
- EU sales — продажи в Европе (в миллионах проданных копий).
- JP sales — продажи в Японии (в миллионах проданных копий).
- Other sales — продажи в других странах (в миллионах проданных копий).
- Critic Score — оценка критиков (от 0 до 100).
- User Score — оценка пользователей (от 0 до 10).
- Rating — рейтинг организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

### Содержимое проекта

1. Загрузка данных и знакомство с ними
2. Проверка ошибок в данных и их предобработка
- Названия, или метки, столбцов датафрейма
- Типы данных
- Наличие пропусков в данных
- Явные и неявные дубликаты в данных
3. Фильтрация данных
4. Категоризация данных

## 1. Загрузка данных и знакомство с ними

Загрузим необходимые библиотеки для анализа данных `/datasets/new_games.csv`. Затем выведем первые строки и результат метода `info()`.

In [90]:
# Импортируем библиотеку pandas
import pandas as pd

In [91]:
# Выгружаем данные из датасета /datasets/new_games.csv в датафрейм games
games = pd.read_csv('https://code.s3.yandex.net/datasets/new_games.csv')

In [92]:
# Выводим информацию о датафрейме
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


In [93]:
# Выводим первые строки датафрейма на экран
games.head()

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


**Общая характеристика данных:** данные датасета `/datasets/new_games.csv` представлены в 11 столбцах и 16956 строках, которые содержат информацию о компьютерных играх, выпущенных на различных платформах. Данные в целом соответствуют описанию, но требуют значительной предобработки.

**Ключевые наблюдения:**
1. Типы данных: 
Преобладающие типы данных `float64(4)` и `object(7)`, но для корректного анализа необходимо преобразовать часть столбцов:
- `Year of Release` - сейчас `float64`. Так как это годы, а не даты, переводить в тип даты `datetime` не имеет смысла, поэтому этот столбец необходимо перевести в тип данных `int64`. 
- Столбцы `Platform`, `Genre` и `Rating` следует преобразовать из `object` в `category`для оптимизации памяти и удобства работы с ограниченным набором значений.
- `EU sales` и `JP sales` - сейчас `object`, но должны быть числовыми `float64`, т.к.данные о продажах представлены в миллионах проданных копий.
- Тип данных в столбцах, содержащих оценку критиков также следует изменить. В `User Score` оценка пользователей (от 0 до 10), что говорит о возможности дробной оценки, поэтому лучше преобразовать в тип данных `float64` из типа `object`.

2. Пропущенные значения:
Значительное количество пропусков встречается в столбцах `Critic Score` и `User Score` более 50%, а в `Rating` около 40%.

**Особенности данных:**
- Названия столбцов содержат пробелы.
- Присутствуют игры, выпущенные до 2000 года.

Исследование данных вывело необходимость значительной предобработки данных для корректного анализа. Большинство столбцов требуют преобразования типов данных. Особое внимание следует уделить обработке многочисленных пропусков в оценочных столбцах, которые носят системный характер и возможно связаны с популярностью игр.

Для дальнейшего анализа необходимо также выполнить следующие ключевые действия: привести названия столбцов к snake_case, преобразовать типы данных в соответствии с их содержанием, обработать пропуски и выполнить фильтрацию данных по требуемому периоду 2000-2013 годов.

---

## 2.  Проверка ошибок в данных и их предобработка


### 2.1. Названия, или метки, столбцов датафрейма

В начале предобработки необходимо привести в порядок названия столбцов.

In [94]:
# Выводим на экран названия всех столбцов датафрейма
games.columns

Index(['Name', 'Platform', 'Year of Release', 'Genre', 'NA sales', 'EU sales',
       'JP sales', 'Other sales', 'Critic Score', 'User Score', 'Rating'],
      dtype='object')

In [95]:
#Приводим все столбцы к стилю snake case
games.columns = games.columns.str.lower().str.replace(' ', '_')

In [96]:
#Проверяем новые названия столбцов
games.columns

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')

### 2.2. Типы данных

Некорректные типы данных могут встречаться из-за наличия пропусков, текстовых значений вместо числовых, либо из-за автоматического приведения Pandas к float, чтобы поддерживать NaN. Также причиной может быть некорректное считывание исходного файла, когда все значения загружаются как строки.
Поэтому необходимо провести преобразования типов данных для корректного анализа.

In [97]:
# Преобразуем продажи в eu_sales, jp_sales и user_score к float64
# ошибки преобразования заменим на NaN
for column in ['eu_sales', 'jp_sales', 'user_score']:
    games[column] = pd.to_numeric(games[column], errors = 'coerce')

In [98]:
#Преобразуем platform, genre и rating к категориальному типу
games = games.astype({
    'platform': 'category',
    'genre': 'category',
    'rating': 'category'
})

In [99]:
# Проверим преобразования с помощью dtypes
games.dtypes

name                 object
platform           category
year_of_release     float64
genre              category
na_sales            float64
eu_sales            float64
jp_sales            float64
other_sales         float64
critic_score        float64
user_score          float64
rating             category
dtype: object

На данном этапе были преобразованы 3 столбца к типу `float64` и 3 столбца к типу `category`. В `year_of_release` мы оставили `float64`, что необходимо для корректного хранения пропущенных значений как NaN. Однако после обработки пропусков мы преобразуем их к целочисленному типу int.

### 2.3. Наличие пропусков в данных

In [100]:
#Посчитаем количество пропусков в каждом столбце
absolute_nulls = games.isna().sum()
absolute_nulls

name                  2
platform              0
year_of_release     275
genre                 2
na_sales              0
eu_sales              6
jp_sales              4
other_sales           0
critic_score       8714
user_score         9268
rating             6871
dtype: int64

In [101]:
# Находим долю пропусков с использованием mean() и сортируем по убыванию
relative_nulls = (games.isna().mean() * 100).round(2).sort_values(ascending=False)
relative_nulls

user_score         54.66
critic_score       51.39
rating             40.52
year_of_release     1.62
eu_sales            0.04
jp_sales            0.02
name                0.01
genre               0.01
platform            0.00
na_sales            0.00
other_sales         0.00
dtype: float64

Исходя из результатов подсчётов, столбцы с пропусками можно разделить на несколько категорий:
1. Критически высокий уровень пропусков: `critic_score` 51.39%, `user_score` 54.66%, `rating` 40.52%. 
- `critic_score` и `user_score` содержат оценки: игры могли не получать профессиональных оценок или пользовательских рецензий по причине низкой популярности, ограниченного релиза или раннего периода выпуска.
- `rating` рейтинг организации ESRB: причиной может быть отсутствие официальной сертификации для некоторых игр.

Скорее всего эти пропуски **не являются ошибочными**.

2. Умеренный уровень пропусков: год выпуска `year_of_release` 1.62%: возможны ошибки в исходных данных или отсутствие информации о дате релиза для редких игр. Т.к. это точная величина, медиана исказит данные, поэтому лучше их просто удалить.
3. Минимальный уровень пропусков (менее 1%): в `name`, `genre`, `eu_sales`, `jp_sales`. Вероятно, причиной пропусков случайные ошибки при сборе данных, их можно удалить.

Удалим пропуски в столбце `year_of_release`и в столбцах, где их процент минимален.

In [102]:
# Удаляем пропуски
games = games.dropna(subset=['year_of_release', 'name', 'genre', 'eu_sales', 'jp_sales'])

In [103]:
#Меняем тип данных на int16
games['year_of_release'] = games['year_of_release'].astype('int16')

В столбцах `critic_score`, `user_score`, `rating` пропуски относятся к типу MNAR. **Их отсутствие может быть связано с объективными факторами, такими как непопулярность игр, ограниченный релиз или отсутствие официального рейтинга.** Для сохранения информативности данных и предотвращения искажения результатов анализа принято решение заменить пропуски в числовых столбцах `critic_score` и `user_score` значением -1, которое выходит за пределы их нормальных диапазонов (0-100 и 0-10 соответственно), что позволит четко идентифицировать такие случаи при анализе. 

В `rating` преобразуем пропуски в явную категорию "Not Rated", что позволит сохранить информативность отсутствия рейтинга как значимого признака для анализа и учитывать такие игры как отдельную группу.

In [104]:
# Заменяем пропуски в числовых столбцах critic_score и user_score на -1
games['critic_score'] = games['critic_score'].fillna(-1)
games['user_score'] = games['user_score'].fillna(-1)

In [105]:
# Добавляем новую категорию к существующим в rating
games['rating'] = games['rating'].cat.add_categories('Not Rated')

In [106]:
#Заменяем пропуски на новую категорию
games['rating'] = games['rating'].fillna('Not Rated')

In [107]:
#Проверяем датасет на наличие пропусков
games.isna().sum()

name               0
platform           0
year_of_release    0
genre              0
na_sales           0
eu_sales           0
jp_sales           0
other_sales        0
critic_score       0
user_score         0
rating             0
dtype: int64

Теперь данные очищены от пропусков и готовы для выполнения следующих этапов анализа.

### 2.4. Явные и неявные дубликаты в данных

Перед поиском явных дубликатов необходимо провести анализ и нормализацию текстовых данных.

In [108]:
#Исследуем столбик platform на неявные дубликаты
unique_platform=games['platform'].unique()
unique_platform

['Wii', 'NES', 'GB', 'DS', 'X360', ..., 'NG', 'TG16', '3DO', 'GG', 'PCFX']
Length: 31
Categories (31, object): ['Wii', 'NES', 'GB', 'DS', ..., 'TG16', '3DO', 'GG', 'PCFX']

In [109]:
#Исследуем столбик genre на неявные дубликаты
unique_genre=games['genre'].unique()
unique_genre

['Sports', 'Platform', 'Racing', 'Role-Playing', 'Puzzle', ..., 'PLATFORM', 'ADVENTURE', 'SIMULATION', 'PUZZLE', 'STRATEGY']
Length: 24
Categories (24, object): ['Sports', 'Platform', 'Racing', 'Role-Playing', ..., 'ADVENTURE', 'SIMULATION', 'PUZZLE', 'STRATEGY']

In [110]:
#Исследуем столбик rating на неявные дубликаты
unique_rating=games['rating'].unique()
unique_rating

['E', 'Not Rated', 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP']
Categories (9, object): ['E', 'Not Rated', 'M', 'T', ..., 'K-A', 'AO', 'EC', 'RP']

Приведем названия платформ и жанров к нижнему регистру, а рейтинги - к верхнему, а также удалим лишние пробелы.
Также заменим неакктуальное название рейтинга на новый.

In [111]:
# Приводим названия платформ и жанров к нижнему регистру и удаляем лишние пробелы
games['platform'] = games['platform'].str.lower().str.strip()  
games['genre'] = games['genre'].str.lower().str.strip()
# Приводим рейтинги к верхнему регистру и заменяем 'К-А' на 'Е'
games['rating'] = games['rating'].str.upper().str.replace('K-A', 'E')

После нормализации текстовых данных можно приступать к поиску явных дубликатов. Нормализация позволила устранить неявные дубликаты, вызванные разным написанием одних и тех же значений, что обеспечит более точное выявление настоящих дубликатов в данных при использовании метода `duplicated()`.

In [112]:
#Сортируем датафрейм по всем столбцам
games_sorted = games.sort_values(by=games.columns.tolist())

In [113]:
#Находим явные дубликаты
dublicates= games_sorted[games_sorted.duplicated(keep=False)]
dublicates

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
15191,Beyblade Burst,3ds,2016,role-playing,0.00,0.00,0.03,0.00,-1.0,-1.0,NOT RATED
15192,Beyblade Burst,3ds,2016,role-playing,0.00,0.00,0.03,0.00,-1.0,-1.0,NOT RATED
15301,11eyes: CrossOver,x360,2009,adventure,0.00,0.00,0.02,0.00,-1.0,-1.0,NOT RATED
15302,11eyes: CrossOver,x360,2009,adventure,0.00,0.00,0.02,0.00,-1.0,-1.0,NOT RATED
4860,18 Wheeler: American Pro Trucker,ps2,2001,racing,0.20,0.15,0.00,0.05,61.0,5.7,E
...,...,...,...,...,...,...,...,...,...,...,...
2909,Yu-Gi-Oh! The Falsebound Kingdom,gc,2002,strategy,0.49,0.13,0.07,0.02,-1.0,-1.0,NOT RATED
6695,Zoo Resort 3D,3ds,2011,simulation,0.11,0.09,0.03,0.02,-1.0,-1.0,E
6696,Zoo Resort 3D,3ds,2011,simulation,0.11,0.09,0.03,0.02,-1.0,-1.0,E
8156,Zumba Fitness Rush,x360,2012,sports,0.00,0.16,0.00,0.02,73.0,6.2,E10+


In [114]:
#Считаем количество явных дубликатов в датафрейме
games.duplicated().sum()

235

In [115]:
#Считаем долю явных дубликатов в датафрейме
games.duplicated().sum()/len(games)*100

1.4098026276321314

В результате проверки обнаружено **235 полных дублированных строк** в данных, что состовляет **1.41%** от общего количества. Все эти записи являются абсолютно идентичными по всем столбцам. 

**Дубликаты будут удалены методом drop_duplicates()**, т.к. они представляют собой избыточную информацию и не несут дополнительной ценности для анализа.

In [116]:
#Сохраняем количество строк до удаления дубликатов
before_row_count=games.shape[0]

In [117]:
#Удаляем дубликаты
games_no_dublicates= games_sorted.drop_duplicates()

In [118]:
#Сохраняем количество строк после удаления дубликатов
after_row_count=games_no_dublicates.shape[0]

In [119]:
print(f"Количество строк до удаления дубликатов: {before_row_count}")
print(f"Количество строк после удаления дубликатов: {after_row_count}")

Количество строк до удаления дубликатов: 16669
Количество строк после удаления дубликатов: 16434


В результате предобработки устранены неявные дубликаты через нормализацию текстовых данных и удалены 241 явный дубликат, что составило 1.42% от всего датасета. **Данные приведены к единому стандарту и готовы для дальнейшего анализа.**

---

## 3. Фильтрация данных

Согласно поставленной задаче анализа игровой индустрии начала XXI века, **необходимо выделить данные за период с 2000 по 2013 год включительно.**

In [120]:
#Создаём новый датафрейм с отфильтрованными данными
games_actual=games[(games['year_of_release']>=2000) & (games['year_of_release']<=2013)]

In [121]:
#Вычисляем количество строк для нового датафрейма
games_actual.shape[0]

12971

В результате фильтрации данных по периоду 2000-2013 годы получен датафрейм `games_actual`, содержащий 12971 записей.

---

## 4. Категоризация данных
    
Для выявления тенденций в игровой индустрии проведем категоризацию игр по оценкам пользователей и критиков. Это позволит определить распределение игр по уровню качества, выявить наиболее активные платформы XXI века.

**Разделим пользовательские оценки и оценки критиков по категориям и посчитаем количество игр для каждой из них:**

In [122]:
#Создаём новый датафрейм с отфильтрованными данными и явно создаем копию
games_actual = games[(games['year_of_release'] >= 2000) & (games['year_of_release'] <= 2013)].copy()

In [123]:
# Категоризация пользовательских оценок user_score
# Создаем функцию для точного контроля границ интервалов
def categorize_users(value):
    if value >= 8: 
        return 'высокая оценка'
    elif value >= 3:
        return 'средняя оценка'
    elif value >= 0:
        return 'низкая оценка'
    else: 
        return 'не оценено'

In [124]:
# Применяем функцию к столбцу user_score
games_actual['user_score_category'] = games_actual['user_score'].apply(categorize_users)

/tmp/ipykernel_55/869680638.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_actual['user_score_category'] = games_actual['user_score'].apply(categorize_users)


In [125]:
# Категоризация оценок критиков critic_score
# Создаем функцию для точного контроля границ интервалов
def categorize_critic(value):
    if value >= 80: 
        return 'высокая оценка'
    elif value >= 30:
        return 'средняя оценка'
    elif value >= 0:
        return 'низкая оценка'
    else: 
        return 'не оценено'

In [126]:
#Применяем функцию к столбцу critic_score
games_actual['critic_score_category'] = games_actual['critic_score'].apply(categorize_critic)

/tmp/ipykernel_55/3276004411.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_actual['critic_score_category'] = games_actual['critic_score'].apply(categorize_critic)


In [127]:
#Проверяем результат
games_actual.head(10)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,user_score_category,critic_score_category
0,Wii Sports,wii,2006,sports,41.36,28.96,3.77,8.45,76.0,8.0,E,высокая оценка,средняя оценка
2,Mario Kart Wii,wii,2008,racing,15.68,12.76,3.79,3.29,82.0,8.3,E,высокая оценка,высокая оценка
3,Wii Sports Resort,wii,2009,sports,15.61,10.93,3.28,2.95,80.0,8.0,E,высокая оценка,высокая оценка
6,New Super Mario Bros.,ds,2006,platform,11.28,9.14,6.50,2.88,89.0,8.5,E,высокая оценка,высокая оценка
7,Wii Play,wii,2006,misc,13.96,9.18,2.93,2.84,58.0,6.6,E,средняя оценка,средняя оценка
8,New Super Mario Bros. Wii,wii,2009,platform,14.44,6.94,4.70,2.24,87.0,8.4,E,высокая оценка,высокая оценка
10,Nintendogs,ds,2005,simulation,9.05,10.95,1.93,2.74,-1.0,-1.0,NOT RATED,не оценено,не оценено
11,Mario Kart DS,ds,2005,racing,9.71,7.47,4.13,1.90,91.0,8.6,E,высокая оценка,высокая оценка
13,Wii Fit,wii,2007,sports,8.92,8.03,3.60,2.15,80.0,7.7,E,средняя оценка,высокая оценка
14,Kinect Adventures!,x360,2010,misc,15.00,4.89,0.24,1.69,61.0,6.3,E,средняя оценка,средняя оценка


Все игры успешно распределены по заданным категориям как для пользовательских оценок, так и для оценок критиков, что позволяет проводить дальнейший анализ предпочтений игроков и критиков.

In [128]:
#Считаем количество игр по категориям пользовательских оценок
user_score_counts = games_actual['user_score_category'].value_counts().reset_index()
#Задаём названия столбцов
user_score_counts.columns = ['user_score_category', 'games_count']
#Сортируем данные по убыванию
user_score_counts = user_score_counts.sort_values('games_count', ascending=False)

In [129]:
#Выводим категории пользовательских оценок с количеством игр
user_score_counts

,user_score_category,games_count
0,не оценено,6406
1,средняя оценка,4145
2,высокая оценка,2303
3,низкая оценка,117


In [130]:
# Считаем количество игр по категориям оценок критиков
critic_score_counts = games_actual['critic_score_category'].value_counts().reset_index()
#Задаём названия столбцов
critic_score_counts.columns = ['critic_score_category', 'games_count']
#Сортируем данные по убыванию
critic_score_counts = critic_score_counts.sort_values('games_count', ascending=False)

In [131]:
#Выводим категории оценок критиков с количеством игр
critic_score_counts

,critic_score_category,games_count
0,не оценено,5711
1,средняя оценка,5499
2,высокая оценка,1706
3,низкая оценка,55


In [132]:
# Считаем количество игр, где обе оценки отсутствуют
both_missing = games_actual[
    (games_actual['user_score_category'] == 'не оценено') & 
    (games_actual['critic_score_category'] == 'не оценено')
].shape[0]
both_missing

5328

В результате анализа распределения игр по категориям оценок, выявлено, что:
- По пользовательским оценкам: наибольшее количество игр получили средние оценки (4145), тогда как низко оцененных игр оказалось совсем немного (117)
- По оценкам критиков: распределение более сбалансировано, с преобладанием игр со средними оценками (5499)
- Особого внимания заслуживает факт, что 5328 игр не были оценены ни пользователями, ни критиками, что может указывать на их низкую популярность или ограниченный релиз.

In [133]:
# Определяем топ-7 платформ по количеству выпущенных игр за период 2000-2013
top_platforms = games_actual['platform'].value_counts().head(7).reset_index()
top_platforms.columns = ['platform', 'games_count']
top_platforms = top_platforms.sort_values('games_count', ascending=False)
top_platforms

,platform,games_count
0,ps2,2153
1,ds,2143
2,wii,1294
3,psp,1198
4,x360,1135
5,ps3,1107
6,gba,825


Были выделены наиболее продуктивные платформы за период 2000-2013 годов. Данные показывают явное доминирование платформ `ps2` (2153 игр) и `ds`(2143 игр) в рассматриваемый период.

---

## 5. Итоговый вывод

В результате проведенной работы был успешно подготовлен аналитический срез данных об играх за период 2000-2013 годов, содержащий 12971 записей.

- Проведена типовая коррекция данных: преобразованы 3 столбца к float64 и 3 столбца к category
- Систематизированы пропуски: критические (40-55% в оценках), умеренные (1.6% в годе выпуска) и минимальные (<1%)
- Обработаны пропуски: удаление незначительных и маркировка специальными значениями критических
- Проведена нормализация текстовых данных и устранение 235 дубликата (1.41% от датасета)
- Выполнена категоризация оценок пользователей и критиков по трем уровням качества

Также были добавлены новые поля:

- `user_score_category` - категоризация пользовательских оценок (низкая/средняя/высокая)
- `critic_score_category` - категоризация оценок критиков (низкая/средняя/высокая)

**Ключевые выводы:**
1. Выявлено 5328 игр, не оцененных ни пользователями, ни критиками, что указывает на нишевый характер этих продуктов.
2. Распределение оценок показывает преобладание среднеоцененных игр: 4145 пользовательских и 5499 критических оценок.
3. Доминирующие платформы периода: PS2 (2153 игр) и DS (2143 игры) демонстрируют рынок с ярко выраженным лидерством.

**Полученный датасет готов для глубокого анализа рыночных тенденций**, исследования корреляции между оценками и коммерческим успехом, а также выявления закономерностей развития игровой индустрии начала XXI века.